# Testing

## Imports

In [1]:
# Install requirements if needed
# !pip install beautifulsoup4 lxml

# 1. Import your parser
import sys
import json
import os
from pprint import pprint
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

from semantic_html.parser import parse_note

## Load HTML

In [2]:
# 2. Define a small test HTML string
html = """
<!DOCTYPE html>
<html>
    <head>
        <meta charset="utf-8" />
    </head>
    <body>
        <div class="zotero-notes">
            <div class="zotero-note">
                <h1>Header1</h1>
                <h2>header2</h2>
                <h3>header3</h3>
                <p><code>p.44</code></p>
                <p>comment related to p. 44 and header 3</p>
                <blockquote>
                    <p style="margin-left: 30px;">
                        “<em>M@áan and Susan</em> is a <span style="font-style: italic;">concept</span> like #animal oder #human#oid. <strong><a href="wd:Paris" rel="noopener noreferrer nofollow">Paris</a></strong> is a <em>city</em>. that is
                        testified by <u>wikipedia</u>.”
                    </p>
                </blockquote>
                <p>
                    <span class="citation">(<a href="zotero://select/library/items/KUHYUVEH">“test”</a>)</span>
                </p>
                <h3>header4</h3>
                <p><code>p.54</code></p>
                <p>comment related to p. 54 and header 4</p>
                <blockquote>
                    <p style="margin-left: 30px;">
                        “<em>Woman</em> is a concept. <strong><a href="wd:Rome" rel="noopener noreferrer nofollow">Rome</a></strong> is a <em>city</em>. that is testified by <u>wikipedia</u>.”
                    </p>
                </blockquote>
                <pre>This will be ignored <em>Man22</em></pre>
            </div>
        </div>
    </body>
</html>

"""

## Load Config

In [3]:
metadata = {"prov:wasAttributedTo":{"@id":"https://github.com/ch-sander"}, "prov:wasGeneratedBy": "https://github.com/ch-sander/semantic-html/blob/main/tests/Testing.ipynb"}


# 3. Define your mapping
mapping = {
    # Top-level mapping keys correspond to semantic classes used during parsing.
    # These must match expected parser categories (Note, Document, Annotation, etc.)
    "@type": ["ResearchNote"],  # Default type for the root Note
    "@id": "https://semantic-html.org/note1",

    "Document": {
        # Represents text blocks (e.g. sentences or paragraphs) in the note. Each doc is the reference doc for text positions in the WADM annotations. If no parent doc is found, 
        # the entire note is used.
        "xpath": ["//p"],  # Matches all <p> tags
        "types": ["Doc"]  # This type will be assigned to these nodes
    },

    "Annotation": {
        # Annotations are inline elements that can be nested in Document.
        # Each sub-entry defines a different category of annotation. Their names, scopes, or purposes
        # do not matter for the mapping.

        "Concept": {
            # Marks concepts, e.g. keywords or philosophical terms.
            'xpath': ['//em | //i | //span[contains(@style, "italic")]'],  # e.g. <em>term</em>
            'split': r"\s*(?:,|\band\b)\s*", # pattern to split element text (first)
            # Split at either a comma or the word "and", with optional surrounding whitespace (non-capturing group)
            # 'find': r"\w+", # pattern to extract relevant text (after splitting)
            "types": ["Annotation", "Concept"]
        },
        "Entity": {
            # Marks named entities, typically rendered as bold text.
            "xpath": ["//strong"],  # e.g. <strong>Rome</strong>
            "types": ["Entity"]
        },

        "Entity2": {
            # Matches entities in plain text using regex, e.g. #human# or #animal
            "regex": [r"#\w+#?"],  # Applied before IGNORE rules! Matches '#' + word + optional trailing '#'
            "types": ["Entity_hash"]
        },

        "Reference1": {
            # Marks generic references, e.g. underlined text.
            "xpath": ["//u"],  # e.g. <u>source</u>
            "types": ["Reference"]
        },

        "Reference2": {
            # Marks Zotero-style citations with specific class.
            'xpath': ['//span[@class="citation"]'],
            "types": ["Reference Zotero"]
        }
    },

    "Locator": {
        # Represents location information (e.g. page numbers).
        "xpath": ["//code"],  # e.g. <code>p.44</code>
        "types": ["Page-Locator"]
    },

    "Structure": {
        # Represents document structure and hierarchy (e.g. headings).
        # Used to model logical organization (chapter, section, etc.).
        "xpath": ["//h1", "//h2", "//h3"],
        "types": ["Structure"]
    },

    "Quotation": {
        # Represents quoted text blocks (e.g. citations or sources).
        "xpath": ["//blockquote"],  # e.g. <blockquote>...</blockquote>
        "types": ["Quotation"]
    },

    "IGNORE": {
        # Defines content to be excluded from parsing, both at HTML and text level.

        "xpath": ["//pre"],  # Removes <pre> blocks entirely
        "regex": ["#"]    # Removes standalone '#' characters from visible text
                          # (use with care – affects all visible text nodes)
    },

    # You may define a JSON-LD context here to control URI mappings (there is a reasonable default as fallback)
    # "@context": {
    #     "@vocab": "https://semantic-html.org/vocab#",
    #     "same:as": {"@id": "owl:sameAs", "@type": "@id"},
    #     "locator": {"@id": "in_locator", "@type": "@id"},
    #     "structure": {"@id": "in_structure", "@type": "@id"},
    #     "note": {"@id": "in_note", "@type": "@id"}
    # },
    "metadata": { # Each key is added to each JSON object in the @graph
        "prov:wasAttributedTo": {
            "@id": "https://github.com/ch-sander"
        },
        "prov:wasGeneratedBy": "https://github.com/ch-sander/semantic-html/blob/main/tests/Testing.ipynb"
    },
    "wadm": { # additional metadata and context for the WADM JSON-LD
        "metadata": {
            "creator": "https://github.com/ch-sander"
        },
        "@context": "https://www.w3.org/ns/anno.jsonld"
    }        
}

## Parse

In [4]:
result = parse_note(html, 
                    mapping, 
                    rdfa=True,
                    wadm=True,
                    tei=True
                    # note_uri="https://semantic-html.org/note1", # here given as '@id' key in mapping
                    # metadata=metadata # here included in in mapping
                    )

## Results

### JSON-LD

In [5]:
print(json.dumps(result["JSON-LD"], indent=4, ensure_ascii=False))

{
    "@context": {
        "xsd": "http://www.w3.org/2001/XMLSchema#",
        "schema": "http://schema.org/",
        "doco": "http://purl.org/spar/doco/",
        "dcterms": "http://purl.org/dc/terms/",
        "prov": "http://www.w3.org/ns/prov#",
        "owl": "http://www.w3.org/2002/07/owl#",
        "@vocab": "https://semantic-html.org/vocab#",
        "Note": "doco:Document",
        "Structure": "doco:DiscourseElement",
        "Locator": "ex:Locator",
        "Doc": "doco:Section",
        "Annotation": "schema:Comment",
        "Quotation": "doco:BlockQuotation",
        "note": {
            "@id": "inNote",
            "@type": "@id"
        },
        "structure": {
            "@id": "inStructure",
            "@type": "@id"
        },
        "locator": {
            "@id": "hasLocator",
            "@type": "@id"
        },
        "sameAs": {
            "@id": "owl:sameAs",
            "@type": "@id"
        },
        "doc": {
            "@id": "dcterms:isPartOf",

#### WADM JSON-LD

In [6]:
print(json.dumps(result["WADM"], indent=4, ensure_ascii=False))

{
    "@context": "https://www.w3.org/ns/anno.jsonld",
    "@graph": [
        {
            "@type": "Annotation",
            "@id": "urn:uuid:bac7bb7d-076c-4435-98bf-8cea0ae09058",
            "created": "2025-08-04T10:24:59.108847",
            "motivation": "identifying",
            "target": {
                "source": "https://semantic-html.org/note1",
                "selector": {
                    "type": "Choice",
                    "items": [
                        {
                            "type": "TextQuoteSelector",
                            "exact": "Header1",
                            "prefix": "",
                            "suffix": " header2 header3 p.44 comment "
                        },
                        {
                            "type": "TextPositionSelector",
                            "start": 0,
                            "end": 7
                        },
                        {
                            "type": "XPathSelector"

### RDFa

In [ ]:
print(result["RDFa"])

<html xmlns:xsd="http://www.w3.org/2001/XMLSchema#" xmlns:schema="http://schema.org/" xmlns:doco="http://purl.org/spar/doco/" xmlns:dcterms="http://purl.org/dc/terms/" xmlns:prov="http://www.w3.org/ns/prov#" xmlns:owl="http://www.w3.org/2002/07/owl#" vocab="https://semantic-html.org/vocab#">
    <head>
        </head>
    <body>
        <div class="zotero-notes">
            <div class="zotero-note">
                <h1 typeof="Structure">Header1</h1>
                <h2 typeof="Structure">header2</h2>
                <h3 typeof="Structure">header3</h3>
                <p typeof="Doc"><code typeof="Page-Locator">p.44</code></p>
                <p typeof="Doc">comment related to p. 44 and header 3</p>
                <blockquote typeof="Quotation">
                    <p style="margin-left: 30px;" typeof="Doc">
                        “<em typeof="Annotation Concept">M@áan</em> is a <span style="font-style: italic;" typeof="Annotation Concept">concept</span> like <span class="Entity2" d

### TEI

In [ ]:
print(result["TEI"])

<?xml version='1.0' encoding='utf-8'?>
<tei:TEI xmlns:tei="http://www.tei-c.org/ns/1.0">
  <tei:teiHeader/>
  <tei:text>
    <tei:body>
      <tei:div xml:id="urn:uuid:a99797e5-4775-4cac-b877-18a0294b2d5a">
        <tei:head xml:id="urn:uuid:a99797e5-4775-4cac-b877-18a0294b2d5a">Header1</tei:head>
        <tei:div xml:id="urn:uuid:a8aa38fe-a3ed-45c6-bbb2-e4b1f729b309">
          <tei:head xml:id="urn:uuid:a8aa38fe-a3ed-45c6-bbb2-e4b1f729b309">header2</tei:head>
          <tei:div xml:id="urn:uuid:1419129f-5ea1-4d47-8ecc-29e46a96c326">
            <tei:head xml:id="urn:uuid:1419129f-5ea1-4d47-8ecc-29e46a96c326">header3</tei:head>
            <tei:milestone unit="page" n="p.44" xml:id="urn:uuid:26ae4622-f847-4281-a1b4-7f776198f834"/>
            <tei:quote xml:id="urn:uuid:dd592e50-99c9-4b2b-b4e4-ee4e90fbc7ae">“M@áan is a concept like animal oder humanoid. Paris is a city. that is testified by wikipedia.”</tei:quote>
            <tei:p xml:id="urn:uuid:0443346a-81f9-483f-9f4a-8f4b0a2cf4b